In [ ]:
!pip install transformers torch pandas scikit-learn requests

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
!pip install transformers torch pandas openpyxl

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd
import requests
from sklearn.model_selection import train_test_split
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Sigma/financial-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("Sigma/financial-sentiment-analysis")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/854 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    prediction = torch.argmax(probabilities)
    confidence = probabilities[0, prediction].item()
    sentiments = ["Negative", "Neutral", "Positive"]
    print (f"Sentiment: {sentiments[prediction]}, Confidence: {confidence:.2f}")

In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    prediction = torch.argmax(probabilities)
    confidence = probabilities[0, prediction].item()
    sentiments = ["Negative", "Neutral", "Positive"]
    return sentiments[prediction], confidence


In [ ]:
predict_sentiment("Given the recent downturn in market conditions and the underperformance of key product lines highlighted in the quarterly report, can you explain what strategies are being implemented to address these challenges?")

('Negative', 0.8546777367591858)

In [ ]:

import pandas as pd

# Load the data from an Excel file
df = pd.read_excel('/content/Earnings call.xlsx')

# predict sentiment for each question and answer
df['Question Sentiment'], df['Question Confidence'] = zip(*df['Question'].apply(lambda x: predict_sentiment(x)))
df['Answer Sentiment'], df['Answer Confidence'] = zip(*df['Answer'].apply(lambda x: predict_sentiment(x)))
print(df.head())



   Year  Quarter                                           Question  \
0  2021        1  It's Martin from Citi. So a question on the Pr...   
1  2021        1  And in terms of the shape of that when it come...   
2  2021        1  I have actually 2 questions, if I may. One, re...   
3  2021        1  \nAnd maybe a follow-up on Timo's answer regar...   
4  2021        1  I was under the impression that some of the ch...   

                                              Answer  \
0  Yes, I think when it comes to Process Automati...   
1  No, I don't think we'll see a kick back like a...   
2  Yes. Guillermo. On the bridge, when you look a...   
3  Well, if we look at Q2, as we say, we expect m...   
4  I think it varies a little bit between our div...   

                                 Category  Category Number Question Sentiment  \
0   Operational Efficiency and Production              4.0           Negative   
1  Financial Performance and Expectations              3.0            Neut

   Year  Quarter                                           Question  \
0  2021        1  It's Martin from Citi. So a question on the Pr...   
1  2021        1  And in terms of the shape of that when it come...   
2  2021        1  I have actually 2 questions, if I may. One, re...   
3  2021        1  \nAnd maybe a follow-up on Timo's answer regar...   
4  2021        1  I was under the impression that some of the ch...   
5  2021        1  I've got 2 questions, please. So on price cost...   
6  2021        1  I wanted to ask 2 questions on the portfolio. ...   
7  2021        1  My second question relates to the other 3 dive...   
8  2021        1  Given how significant some of the kind of comp...   
9  2021        1  And a couple of follow-ups. I mean just in ter...   

                                              Answer  \
0  Yes, I think when it comes to Process Automati...   
1  No, I don't think we'll see a kick back like a...   
2  Yes. Guillermo. On the bridge, when you look a.

In [ ]:
# load the data to an Excel file
df.to_excel('EarningsCallWithSentiments.xlsx', index=False)